In [1]:
import pandas as pd
import numpy as np
import os
from scipy.stats import norm

# --- 1. Setup ---
DATA_DIR = "data"
# Load individual asset returns
asset_returns = pd.read_csv(
    os.path.join(DATA_DIR, "asset_log_returns.csv"),
    index_col='Date',
    parse_dates=True
)
# Define current portfolio weights (equal weight) and confidence level
weights = np.array([1/asset_returns.shape[1]] * asset_returns.shape[1])
alpha = 0.01 # For 99% VaR

print("✅ Setup complete. Asset returns loaded.")

# --- 2. Calculate Portfolio Statistics ---
# Use the full historical covariance matrix and mean returns
cov_matrix = asset_returns.cov()
mean_returns = asset_returns.mean()
portfolio_mean_return = mean_returns.dot(weights)
portfolio_volatility = np.sqrt(weights.T @ cov_matrix @ weights)
z_score = norm.ppf(alpha)

# Calculate total Parametric VaR (expressed as a negative return)
total_var = portfolio_mean_return + portfolio_volatility * z_score

print(f"\nPortfolio Mean Return (Daily): {portfolio_mean_return*100:.4f}%")
print(f"Portfolio Volatility (Daily): {portfolio_volatility*100:.4f}%")
print(f"Z-score for {1-alpha:.0%} VaR: {z_score:.4f}")
print("-" * 40)
print(f"Total Portfolio 99% Parametric VaR: {total_var*100:.4f}%")
print("-" * 40)

✅ Setup complete. Asset returns loaded.

Portfolio Mean Return (Daily): 0.0446%
Portfolio Volatility (Daily): 1.0446%
Z-score for 99% VaR: -2.3263
----------------------------------------
Total Portfolio 99% Parametric VaR: -2.3856%
----------------------------------------


In [2]:
# Beta_i = Cov(Asset_i, Portfolio) / Var(Portfolio)
# Cov(Asset_i, Portfolio) is the i-th element of (cov_matrix @ weights)
beta_to_portfolio = (cov_matrix @ weights) / (portfolio_volatility**2)

print("--- Asset Betas to Portfolio ---")
print(beta_to_portfolio.round(4))

# --- 2. Calculate Marginal VaR (MVaR) ---
# We assume 'z_score' is in memory from sub-step 1
# MVaR_i = z_score * Cov(Asset_i, Portfolio) / Portfolio_Volatility
marginal_var = z_score * (cov_matrix @ weights) / portfolio_volatility

# Convert MVaR to percentage terms for easier interpretation
mvar_percent = marginal_var * 100

print("\n--- Marginal VaR (MVaR) ---")
print(mvar_percent.round(4).to_frame(name='MVaR (%)'))

--- Asset Betas to Portfolio ---
AAPL    1.3912
GE      1.4668
GLD     0.1215
HYG     0.4462
JPM     1.6646
MSFT    1.2925
NVDA    2.2099
QQQ     1.2354
SPY     1.1298
TLT    -0.2430
UUP    -0.0922
XLE     1.3772
dtype: float64

--- Marginal VaR (MVaR) ---
      MVaR (%)
AAPL   -3.3809
GE     -3.5647
GLD    -0.2952
HYG    -1.0844
JPM    -4.0452
MSFT   -3.1411
NVDA   -5.3706
QQQ    -3.0023
SPY    -2.7456
TLT     0.5906
UUP     0.2241
XLE    -3.3469


In [4]:
# CVaR_i = MVaR_i * weight_i
component_var = marginal_var * weights

# Convert CVaR to percentage terms
cvar_percent = component_var * 100

print("--- Component VaR (CVaR) ---")
print(cvar_percent.round(4).to_frame(name='CVaR (%)'))


# --- 2. Calculate Percentage Contribution ---
# We assume 'total_var' (scalar, negative decimal return) is in memory.
# Convert total_var to a positive number for this calculation
total_var_positive = -total_var

# % Contribution = (CVaR_i / Total_VaR) * 100
# Note: CVaR is negative, Total VaR is positive, so % Contribution is negative.
# We multiply by -1 to show contribution as a positive percentage.
percent_contribution = (-component_var / total_var_positive) * 100

print("\n--- Percentage Contribution to VaR ---")
print(percent_contribution.round(2).to_frame(name='Contribution (%)'))

# --- Sanity Check: Sum of Contributions ---
# The sum of individual Component VaRs should equal the total portfolio VaR.
# The sum of percentage contributions should equal 100%.
print("\n--- Sanity Checks ---")
print(f"Sum of Component VaRs: {-component_var.sum()*100:.4f}% (Should match Total VaR: {-total_var*100:.4f}%)")
print(f"Sum of Percent Contributions: {percent_contribution.sum():.2f}% (Should be 100% +/- 2%)")

--- Component VaR (CVaR) ---
      CVaR (%)
AAPL   -0.2817
GE     -0.2971
GLD    -0.0246
HYG    -0.0904
JPM    -0.3371
MSFT   -0.2618
NVDA   -0.4475
QQQ    -0.2502
SPY    -0.2288
TLT     0.0492
UUP     0.0187
XLE    -0.2789

--- Percentage Contribution to VaR ---
      Contribution (%)
AAPL             11.81
GE               12.45
GLD               1.03
HYG               3.79
JPM              14.13
MSFT             10.97
NVDA             18.76
QQQ              10.49
SPY               9.59
TLT              -2.06
UUP              -0.78
XLE              11.69

--- Sanity Checks ---
Sum of Component VaRs: 2.4302% (Should match Total VaR: 2.3856%)
Sum of Percent Contributions: 101.87% (Should be 100% +/- 2%)
